### Implementing Data Validation using Protobuf in a Distributed System
**Description**: Use gRPC to implement a distributed system that validates messages using
Protobuf.

**Steps**:
1. Create a .proto file for gRPC service.
2. Implement server-side validation
    - Create a gRPC server
    - Bind the server to an address
    - Start server

In [1]:
# Write your code from here

from google.protobuf import descriptor_pb2, descriptor_pool, message_factory
import pandas as pd

# --- 1. Dynamically Define a Protobuf Schema via DescriptorProto ---

file_desc = descriptor_pb2.FileDescriptorProto()
file_desc.name = 'customer.proto'
file_desc.package = 'customer_package'
msg_type = file_desc.message_type.add()
msg_type.name = 'Customer'

# Define fields: (name, number, type)
fields = [
    ('customer_id', 1, descriptor_pb2.FieldDescriptorProto.TYPE_INT32),
    ('name',        2, descriptor_pb2.FieldDescriptorProto.TYPE_STRING),
    ('email',       3, descriptor_pb2.FieldDescriptorProto.TYPE_STRING),
    ('balance',     4, descriptor_pb2.FieldDescriptorProto.TYPE_FLOAT),
    ('is_active',   5, descriptor_pb2.FieldDescriptorProto.TYPE_BOOL),
]

for name, num, ftype in fields:
    fld = msg_type.field.add()
    fld.name   = name
    fld.number = num
    fld.label  = descriptor_pb2.FieldDescriptorProto.LABEL_OPTIONAL
    fld.type   = ftype

# Register in a Pool and get a Python class
pool = descriptor_pool.DescriptorPool()
file_md = pool.Add(file_desc)
cust_desc = pool.FindMessageTypeByName('customer_package.Customer')

# Correctly create the prototype (message class)
CustomerMessage = message_factory.MessageFactory(pool).CreatePrototype(cust_desc)

# --- 2. Sample Records to Validate ---

records = [
    {"customer_id": 1, "name": "Alice",   "email": "alice@example.com", "balance": 120.0, "is_active": True},
    {"customer_id": 2, "name": "Bob",     "email": "bob@example.com",   "balance":  75.5, "is_active": False},
    {"customer_id": 3, "name": "",        "email": "charlie@",         "balance": -10.0, "is_active": True},  # invalid
    {"customer_id": 4, "name": "Dana",    "email": "dana@example.com",  "balance": "n/a", "is_active": True},  # invalid
]

# --- 3. Validation & Encoding Loop ---

valid = []
invalid = []

for rec in records:
    try:
        # Attempt to construct + serialize the message
        msg = CustomerMessage(**rec)
        # double‐check types by serializing to bytes
        _ = msg.SerializeToString()
        valid.append(rec)
    except Exception as e:
        invalid.append((rec, str(e)))

# --- 4. Display Results ---

print("✅ Valid Records:")
for v in valid:
    print(" ", v)

print("\n❌ Invalid Records:")
for v, err in invalid:
    print(" ", v, "\n    →", err)

# --- 5. Build a DataFrame of Only the Valid Records ---

df_valid = pd.DataFrame(valid)
print("\n📘 Valid Records DataFrame:")
print(df_valid)

AttributeError: 'MessageFactory' object has no attribute 'CreatePrototype'